In [74]:
library(dplyr)
library(tidyverse)

library(ggplot2)
library(cowplot)

library(readxl)

options(repr.plot.width=15, repr.plot.height=6)

In [165]:
df <- read.csv("df_n_frutos_cum.csv")

df$bioproduto <- ifelse(df$tratamento <= 2, 0.5,
    ifelse(df$tratamento <= 4,1,
        ifelse(df$tratamento <= 6,1.5,
            ifelse(df$tratamento <= 8,2,
                ifelse(df$tratamento <= 10,2.5,
                    ifelse(df$tratamento == 11,5,
                        10
                    )
                )
            )
        )
    )
)
df$alternado <- factor(1 + df$tratamento %% 2)

df$tratamento <- factor(df$tratamento)
df$bloco <- factor(df$bloco)

head(df, 6)

,tratamento,bloco,colheita,n_frutos,n_frutos_cum,peso,comprimento,largura,bioproduto,alternado
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,1,1,1,1.5,1.5,309.0,62.33333,82.33333,0.5,2
2,1,1,2,6.5,8.0,1489.0,60.76923,77.23077,0.5,2
3,1,1,3,1.5,9.5,340.5,51.00000,79.00000,0.5,2
4,1,1,4,7.0,16.5,1288.0,54.64286,71.14286,0.5,2
5,1,1,5,10.0,26.5,2119.0,64.10000,77.00000,0.5,2
6,1,1,6,3.0,29.5,785.0,70.66667,85.00000,0.5,2


In [202]:
contr.sum

function (n, contrasts = TRUE, sparse = FALSE) 
{
    if (length(n) <= 1L) {
        if (is.numeric(n) && length(n) == 1L && n > 1L) 
            levels <- seq_len(n)
        else stop("not enough degrees of freedom to define contrasts")
    }
    else levels <- n
    levels <- as.character(levels)
    cont <- .Diag(levels, sparse = sparse)
    if (contrasts) {
        cont <- cont[, -length(levels), drop = FALSE]
        cont[length(levels), ] <- -1
        colnames(cont) <- NULL
    }
    cont
}
<bytecode: 0x55984b4f2df8>
<environment: namespace:stats>

In [201]:
df_design <- df %>% group_by(bioproduto, alternado, bloco) %>%
summarise(
    # tratamento =
    last_n_frutos_cum = list(n_frutos_cum[8:10])
) %>% unnest(last_n_frutos_cum)

df_design <- df_design[1:90,]
df_design$bioproduto <- factor(df_design$bioproduto)

contrasts(df_design$bioproduto) <- contr.sum(5, contrasts=TRUE)
fit <- lm(last_n_frutos_cum ~ bioproduto + alternado + bioproduto:alternado + bloco, data = df_design)
s <- summary(fit)

contrasts(df_design$bioproduto) <- contr.sum(levels(df_design$bioproduto))
fit <- lm(last_n_frutos_cum ~ bioproduto + alternado + bloco, data = df_design)
s <- summary(fit)
s

`summarise()` has grouped output by 'bioproduto', 'alternado'. You can override using the `.groups` argument.



Call:
lm(formula = last_n_frutos_cum ~ bioproduto + alternado + bloco, 
    data = df_design)

Residuals:
    Min      1Q  Median      3Q     Max 
-17.144  -4.774  -1.039   5.090  15.000 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   35.489      1.601  22.173   <2e-16 ***
bioproduto1   -3.844      1.601  -2.402   0.0186 *  
bioproduto2    2.183      1.601   1.364   0.1763    
bioproduto3    2.239      1.601   1.399   0.1656    
bioproduto4    1.100      1.601   0.687   0.4939    
alternado2    -2.144      1.601  -1.340   0.1840    
bloco2         3.400      1.960   1.734   0.0866 .  
bloco3         1.633      1.960   0.833   0.4071    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 7.592 on 82 degrees of freedom
Multiple R-squared:  0.1434,	Adjusted R-squared:  0.07032 
F-statistic: 1.962 on 7 and 82 DF,  p-value: 0.07032


In [141]:
model <- aov(last_n_frutos_cum ~ bioproduto + alternado + bioproduto:alternado + bloco, data = df_design)
summary(model)

                     Df Sum Sq Mean Sq F value Pr(>F)  
bioproduto            4    515  128.63   2.247 0.0715 .
alternado             1    103  103.47   1.807 0.1827  
bloco                 2    173   86.74   1.515 0.2262  
bioproduto:alternado  4    261   65.23   1.139 0.3443  
Residuals            78   4466   57.25                 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [142]:
a <- 5
b <- 2
delta <- 3
n <- 3

In [136]:
j = 1
df_design[df_design$alternado == j,]$last_n_frutos_cum

[1] 14.5 16.0 25.0 36.5 37.5 39.5 29.5 41.5 47.5 33.5 39.0 43.5 33.5 39.0 46.5
[16] 37.5 44.5 53.0 33.5 36.0 40.5 41.0 47.0 52.0 27.5 37.5 42.5 34.0 35.0 43.0
[31] 38.5 43.5 54.5 33.0 36.0 43.0 27.0 29.0 33.0 28.0 33.0 37.0 31.0 35.5 43.5

In [144]:
# SSQ_blocos
sum(sapply(1:delta, function(j){
    sum( df_design[df_design$bloco == j,]$last_n_frutos_cum )
})^2 / (a*b)) - sum(df_design$last_n_frutos_cum)^2/(a*b*n)

# SSQ_alternado
sum(sapply(1:b, function(j){
    sum( df_design[df_design$alternado == j,]$last_n_frutos_cum )
})^2 / (a*n)) - sum(df_design$last_n_frutos_cum)^2/(a*b*n)

[1] 520.4667

[1] 310.4083